In [1]:
%run "KiteConnect_Library.ipynb"   

warnings.filterwarnings('ignore')

mode = "Live" # Backtesting/Live/Algo
offline = False
if mode =="Live":
    offline = False 

In [2]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

portfolio = pd.DataFrame(downloadlist.tolist())
portfolio = pd.DataFrame(['TATAMOTORS','WIPRO','HDFCBANK'])

portfolioToken = portfolio[0].apply(EQTOKEN).tolist()

In [3]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0

# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a, days: ROC(a,days) >= 0.01
#TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        iRSI = RSI(CLOSE, timeperiod=14)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
        #atr = ATR(CLOSE)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def long_macd_crossover(self):
        return pd.DataFrame(CROSSOVER(iMACD, iMACDsignal), columns=["buy"])
    
    
    def short_macd_crossover(self):
        return pd.DataFrame(CROSSOVER(iMACDsignal, iMACD), columns=["sell"])
    
    def long_pa(self):
        return pd.DataFrame(TREND_DOWN(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
                            #| (CDLABANDONEDBABY(OPEN, HIGH, LOW, CLOSE) != 0) 
                            ), columns=["buy"])
    def short_pa(self):
        return pd.DataFrame( TREND_UP(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
                            #| (CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0)
                            ) , columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_pa() # |self.long_macd_crossover() # | self.long_bb() #| self.long_ichimoku() #| self.long_bull_engulf_ema() #| self.long_ichimoku()
        
        sell = self.short_pa() # |self.short_macd_crossover() # | self.short_bb() #| self.short_bear_engulf() #| self.short_hanging_man() #|  self.short_ichimoku()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        buy['sl'] = CLOSE - SD
        sell['sl'] = CLOSE + SD
        buy['so'] = CLOSE + 4 * SD
        sell['so'] = CLOSE - 4 * SD

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    def tradeRecommendation(self):
        buy, sell = self.tradeDecision()
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        logging.debug(str(timeNow))
        logging.debug(buy.tail(3))
        logging.debug(sell.tail(3))
        if (delT1 == buy.index[-1]) | (delT2 == buy.index[-1]) :
            return "buy,"+str(timeNow) +","+str(buy.iloc[-1]['low'])+","+str(buy.iloc[-1]['close'])+"\n"
        elif (delT1 == sell.index[-1]) | (delT2 == sell.index[-1]) :  
            return "sell,"+str(timeNow) +","+str(sell.iloc[-1]['high'])+","+str(sell.iloc[-1]['close'])+"\n"
        return ""

    
    def tradeBot(self, symbol):
        global STATE
        buy, sell = self.tradeDecision()
        logger.debug(symbol+","+STATE)
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        
        price = CLOSE[-1]
        trigger = CLOSE[-1]
        #stoploss = SD[-1]
        stoploss = 0
        squareoff = 3 * SD[-1] 
        
        logger.debug(buy)
        logger.debug(sell)
        
        if buy.empty | sell.empty:
            return
        
        if (buy.empty != True) & ((delT1 == buy.index[-1]) | (delT2 == buy.index[-1])) & (STATE != "SELL"):
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_sl(symbol, price, trigger, quantity=1)
            STATE = "SELL"
        elif (sell.empty != True) & ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) & (STATE != "BUY"):
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_sl(symbol, price, trigger, quantity=1)
            
    def papertradeBot(self, symbol, timestamp):
        global STATE, profit, squareoff, stoploss
        buy, sell = self.tradeDecision()
        #logger.debug(buy)

        timeNow = timestamp
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)

        #logger.debug(timeNow)

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = 2*SD[-1]
        squareoff = 3 * SD[-1] 
        
        if LOW[-1]<=stoploss:
            profit = profit - stoploss
            STATE = "BUY"
            return
        
        #if HIGH[-1] >= squareoff:
        #    profit = profit - squareoff
        #    STATE = "BUY"
        #    return

        if buy.empty | sell.empty:
            return
        
        if ((delT1 == buy.index[-1]) or (delT2 == buy.index[-1])) and (STATE != "SELL"):
            profit = profit - price
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, 0, quantity=1)
            STATE = "SELL"
        elif ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) and (STATE != "BUY"):
            profit = profit + price
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, 0, quantity=1)
            
        
papertrade = False
if (mode=="Algo") & (papertrade == False):
    global fig
    noofcandles = 60
    x = "HDFC"
    temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    
    #logger.debug(temp_data)
    #temp_data = downloadData(x, fromDate, toDate, "day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    #temp_data = aroon(temp_data,5, True)
    temp_data = stoch(temp_data,5, True)
    temp_data = sma(temp_data,1,True)
    obv(temp_data,2, True)
    

    #print(temp_data.tail(200))
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    #print(buy)
    #print(sell)
    #print(CLOSE)
    
    #a.tradeRecommendation()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)


In [4]:
if mode == "Live":
    kws.connect(threaded=True)

In [6]:
if mode == "Live":
    kws.close()

In [7]:
if mode == "Live":
    print(kws.is_connected())

False
